In [1]:
import pandas as pd
import numpy as np

#base de datos con las denuncias
df_paot = pd.read_csv(r"C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\clean\datos_clean.csv", encoding = 'latin')


C:\Users\samue\AppData\Local\Temp\ipykernel_16028\3366487597.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_paot = pd.read_csv(r"C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\clean\datos_clean.csv", encoding = 'latin')


In [2]:
from simpledbf import Dbf5

dbf = Dbf5(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\raw\ri_11.dbf')

#base de datos externa, densidad de poblacion
df_pob = dbf.to_dataframe().dropna()


PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [3]:
df_pob

,cve_ent,alcaldia,cve_col,colonia,pob_2010,SUP_COL_M2,VIV2010,DENVIVHa,SUPMNZHa,C_DENVIVHa
0,09,AZCAPOTZALCO,02-001,AGUILERA,2014.0,103825.0,680,65.4948,10.38250,4
1,09,AZCAPOTZALCO,02-002,ALDANA,3378.0,154812.0,1088,70.2788,15.48120,4
2,09,AZCAPOTZALCO,02-005,ANGEL ZIMBRON,2737.0,242232.0,1067,44.0487,24.22320,3
3,09,AZCAPOTZALCO,02-006,ARENAL,4817.0,286075.0,1493,52.1891,28.60750,3
4,09,AZCAPOTZALCO,02-007,CENTRO DE AZCAPOTZALCO,3043.0,424471.0,1118,26.3387,42.44710,2
...,...,...,...,...,...,...,...,...,...,...
1809,09,ALVARO OBREGON,10-234,UNION POPULAR EMILIANO ZAPATA (U HAB),2618.0,137210.0,796,58.0133,13.72100,3
1810,09,ALVARO OBREGON,10-235,VALENTIN GOMEZ FARIAS,1835.0,113947.0,416,36.5082,11.39470,2
1811,09,ALVARO OBREGON,10-237,VILLA SOLIDARIDAD,588.0,30095.5,189,62.8001,3.00955,4
1812,09,ALVARO OBREGON,10-239,ZENON DELGADO,2658.0,107451.0,720,67.0073,10.74510,4


In [4]:
#creamos dataframes para trabajar el linkage

colonias_paot=pd.DataFrame(columns=['alcaldia','colonia'], index=range(df_paot.groupby(['alcaldia', 'colonia']).size().shape[0]))
colonias_pob = pd.DataFrame(columns=['alcaldia', 'colonia', 'cve_col'], index=range(df_pob.groupby(['alcaldia','colonia']).size().shape[0]))

#se llena el dataframe de colonias_paot
index = 0
for alcaldia in df_paot['alcaldia'].unique():
    for colonia in df_paot[df_paot['alcaldia']==alcaldia]['colonia'].unique():

        colonias_paot.iloc[index, 0] = alcaldia
        colonias_paot.iloc[index, 1] = colonia

        
        index = index +1


index = 0

#se llena el dataframe de colonias_pob
for alcaldia in df_pob['alcaldia'].unique():
    for colonia in df_pob[df_pob['alcaldia']==alcaldia]['colonia'].unique():

        colonias_pob.iloc[index, 0] = alcaldia
        colonias_pob.iloc[index, 1] = colonia
        colonias_pob.iloc[index, 2] = df_pob[(df_pob['alcaldia']==alcaldia) & (df_pob['colonia'] == colonia)].iloc[0,2]
        
        index = index +1
            


colonias_pob


,alcaldia,colonia,cve_col
0,AZCAPOTZALCO,AGUILERA,02-001
1,AZCAPOTZALCO,ALDANA,02-002
2,AZCAPOTZALCO,ANGEL ZIMBRON,02-005
3,AZCAPOTZALCO,ARENAL,02-006
4,AZCAPOTZALCO,CENTRO DE AZCAPOTZALCO,02-007
...,...,...,...
1809,VENUSTIANO CARRANZA,VENUSTIANO CARRANZA (AMPL),17-063
1810,VENUSTIANO CARRANZA,VIADUCTO - BALBUENA (CONJ HAB),17-064
1811,VENUSTIANO CARRANZA,EMILIANO ZAPATA (U HAB),17-024
1812,VENUSTIANO CARRANZA,MICHOACANA (AMPL),17-040


In [5]:
#se limpian ambos dataframes
'''
import recordlinkage as rl
from recordlinkage.preprocessing import clean

#primero con funcion de record linkage (quita acentos, hace minusculas)
colonias_paot['colonia'] = rl.preprocessing.clean(colonias_paot['colonia'], strip_accents='ascii', remove_brackets=False)
colonias_paot['alcaldia'] = rl.preprocessing.clean(colonias_paot['alcaldia'], strip_accents='ascii', remove_brackets=False)
colonias_pob['colonia'] = rl.preprocessing.clean(colonias_pob['colonia'], strip_accents='ascii', remove_brackets=False)
colonias_pob['alcaldia'] = rl.preprocessing.clean(colonias_pob['alcaldia'], strip_accents='ascii', remove_brackets=False)

#luego con funciones de javier (estandariza numeros, quita etiquetas sobrantes)

limpiar_colonias(colonias_paot, 'colonia')
limpiar_colonias(colonias_pob, 'colonia')

'''


"\nimport recordlinkage as rl\nfrom recordlinkage.preprocessing import clean\n\n#primero con funcion de record linkage (quita acentos, hace minusculas)\ncolonias_paot['colonia'] = rl.preprocessing.clean(colonias_paot['colonia'], strip_accents='ascii', remove_brackets=False)\ncolonias_paot['alcaldia'] = rl.preprocessing.clean(colonias_paot['alcaldia'], strip_accents='ascii', remove_brackets=False)\ncolonias_pob['colonia'] = rl.preprocessing.clean(colonias_pob['colonia'], strip_accents='ascii', remove_brackets=False)\ncolonias_pob['alcaldia'] = rl.preprocessing.clean(colonias_pob['alcaldia'], strip_accents='ascii', remove_brackets=False)\n\n#luego con funciones de javier (estandariza numeros, quita etiquetas sobrantes)\n\nlimpiar_colonias(colonias_paot, 'colonia')\nlimpiar_colonias(colonias_pob, 'colonia')\n\n"

In [6]:
import funciones
funciones.estandarizacion(colonias_paot, 'colonia', 'alcaldia')
funciones.estandarizacion(colonias_pob, 'colonia', 'alcaldia')

In [7]:
colonias_pob[colonias_pob['colonia'].str.contains('agricola oriental')]

,alcaldia,colonia,cve_col
409,iztacalco,cuchilla agricola oriental,06-006
419,iztacalco,agricola oriental 1,06-040
420,iztacalco,agricola oriental 2,06-041
421,iztacalco,agricola oriental 3,06-042
422,iztacalco,agricola oriental 4,06-043
423,iztacalco,agricola oriental 5,06-044
424,iztacalco,agricola oriental 6,06-045
425,iztacalco,agricola oriental 7,06-046
426,iztacalco,agricola oriental 8,06-047


In [8]:
#una vez que tenemos ambos dataframes limpios, empezamos con el linkage

#creamos dataframe para ir guardando matches
colonias_matched = pd.DataFrame(columns=['alcaldia', 'colonia_vieja','colonia_nueva'], index=range(df_paot.groupby(['alcaldia', 'colonia']).size().shape[0]))

#creamos lista para ir guardando indices de colonias faltantes

index_faltantes_paot = colonias_paot.index.values.tolist()


In [9]:
#empezamos primer filtro: linkage con matches exactos en colonia y alcaldia
#para entender esto recomendamos leer documentacion de recordlinkage

import recordlinkage as rl

indexer = rl.Index()
indexer.full()
pairs = indexer.index(colonias_pob, colonias_paot)

compare_cl = rl.Compare()
compare_cl.exact("colonia", "colonia", label = 'colonia')
compare_cl.exact("alcaldia", "alcaldia", label='alcaldia')
features = compare_cl.compute(pairs, colonias_pob, colonias_paot)

features[(features['colonia']==1)&(features['alcaldia']==1)]

,,colonia,alcaldia
0,1231,1,1
1,1185,1,1
2,1193,1,1
3,1242,1,1
5,1164,1,1
...,...,...,...
1805,1303,1,1
1806,1342,1,1
1807,1289,1,1
1808,1319,1,1


In [10]:
#actualizamos colonias_matched e index_faltantes_paot con los resultados del linkage
#ojo: es mejor guardar indices en una lista para poder borrarlos mas sencillamente (hablando de index_faltantes_paot)

parejas = features[(features['colonia']==1)&(features['alcaldia']==1)].index

for p in parejas:
    if p[1] in index_faltantes_paot:
        index_faltantes_paot.remove(p[1])
    
    colonias_matched.iloc[p[1], 0] = colonias_paot.iloc[p[1], 0]
    colonias_matched.iloc[p[1], 1] = colonias_paot.iloc[p[1], 1]
    colonias_matched.iloc[p[1], 2] = colonias_pob.iloc[p[0], 1]



In [11]:
#creamos diccionario con las colonias que linkeamos manualmente
#este lo hizo sammy

iztacalco = [('administracion agricola oriental','agricola oriental 3'),
('administracion apatlaco','agricola oriental 3'),
('administracion ramos millan amp','tlacotal ramos millan'),
('administracion santiago sur','santiago sur'),
('adolfo ruiz cortines','adolfo ruiz cortinez 1'),
('agricola oriental','agricola oriental 3'),
('agricola pantitlan','agricola oriental 3'),
('benito juarez','campamento 2 de octubre 1'),
('coyuya','fraccionamiento coyuya'),
('del valle','aricola oriental 1'),
('delegacion politica iztacalco','campamento 2 de octubre 1'),
('el recreo','campamento 2 de octubre 1'),
('exejido magdalena mixhuca','ex ejidos de la magdalena mixihuca'),
('gabriel ramos millan amp','tlacotal ramos millan')]

iztapalapa = [('acapultitlan','paseos de churubusco'),
('administracion ermita','ermita zaragoza 1'),
('administracion ermita zaragoza','ermita zaragoza 1'),
('administracion iztapalapa','la planta'),
('administracion san andres tomatlan','cerro de la estrella 1'),
('administracion santa martha acatitla','sta martha acatitla nte amp 1'),
('administracion vicente guerrero','vicente guerrero super manzana 1'),
('albarrada','albarrada'),
('anahuac','san juan xalpa 1'),
('articulo 4 constitucional','art 4 constitucional'),
('barranca de guadalupe','guadalupe'),
('bellavista amp','bellavista'),
('biatlon','ejercito de oriente 1'),
('buenavista','buenavista 1'),
('cabeza de juarez','cabeza de juarez 1'),
('cabeza de juarez secc 6','cabeza de juarez 3'),
('campestre estrella','guadalupe'),
('carlos jonguitud barrios','loma estrella 3'),
('casa blanca rodeo','casa blanca'),
('celoalliotli','el molino'),
('central de abasto','paseos de churubusco'),
('cerro de la estrella','cerro de la estrella 1'),
('cerro estrella','cerro de la estrella 1'),
('chinampac de juarez','chinampac de juarez 1'),
('chinampac de juarez frente 9','chinampac de juarez 1'),
('chinampac de juarez frente 7','chinampac de juarez 1'),
('chinampac de juarez frente 8','chinampac de juarez 1'),
('chinampac de juarez frente 10','chinampac de juarez 1'),
('chinampac de juarez frente 11','chinampac de juarez 1'),
('complejo industrial ecologico','chinampac de juarez 1'),
('consejo agrarista mexicano','consejo agrarista mexicano 1'),
('constitucion de 1 9 1 7','constitucion de 1917 1'),
('cuchilla del moral 1','cuchillas del moral'),
('cuchilla del moral 2','cuchillas del moral'),
('delegacion politica iztapalapa','san lucas'),
('desarrollo urbano quetzalcoatl','desarrollo urbano quetzalcoatl 1'),
('ejercito constitucionalista','ejto constitucionalista'),
('ejercito de oriente 2','ejto de ote 2'),
('ejercito de oriente vecinal 1','ejto de ote 2'),
('ejercito de oriente vecinal 2','ejto de ote 2'),
('ejercito de oriente zona peñon','ejto de ote 2'),
('el gavilan','gavilan'),
('el molino tezonco','rinconada del molino'),
('el paraiso','san pablo i, 2 y 5 lomas del paraiso'),
('ermita hidalgo','ermita zaragoza 1'),
('ermita iztapalapa','ermita zaragoza 1'),
('ermita zaragoza','ermita zaragoza 1'),
('esmeralda','granjas esmeralda'),
('eva samano de lopez mateos','albarradas'),
('exhacienda san nicolas tolentino','san nicolas tolentino 1'),
('gamma gavilan','gama gavilan'),
('granjas estrella','granjas estrella 1'),
('guelatao benito juarez','guelatao de juarez 1'),
('hidalgo y mina','las americas'),
('huitzico','huitzico la poblanita'),
('infonavit','sta isabel industrial'),
('ing zaragoza','ignacio zaragoza'),
('iztapalapa centro','san lucas'),
('jardines de san lorenzo tezonco','san lorenzo tezonco'),
('jose lopez portillo','jose lopez portillo 1'),
('jose maria morelos y pavon','jose maria morelos y pavon'),
('juan escutia','juan escutia 1'),
('justo sierra maestro','justo sierra'),
('la hera','la era')]

g_a_madero = [('5 de mayo','cuautepec de madero'),
('administracion acueducto','santiago atepetlac'),
('administracion camionera del norte','c t m aragon (u)'),
('administracion cuautepec','cuautepec de madero'),
('administracion gustavo a madero','villa gustavo a madero'),
('administracion pradera','villa gustavo a madero'),
('aguilera','dm nacional'),
('anzures','san juan de aragon'),
('aragon','san juan de aragon'),
('aragon fovissste','san juan de aragon'),
('aragon la villa','san juan de aragon'),
('arboledas','arboledas de cuautepec'),
('arboledas de cuautepec el alto','arboledas de cuautepec'),
('atzacoalco','ctm atzacoalco'),
('atzacoalco ctm','ctm atzacoalco'),
('campestre aragon','campestre aragon 1'),
('casas aleman amp','casas aleman amp 1'),
('castillo el chico','castillo chico'),
('castillo el grande','castillo grande'),
('chalma de guadalupe','chalma de guadalupe 1'),
('del carmen','castillo chico'),
('del carmen cuautepec','castillo chico'),
('del mar','ex escuela de tiro'),
('delegacion politica gustavo a madero','gabriel hernandez'),
('eduardo molina','eduardo molina 1'),
('el arbolillo ctm','el arbolillo'),
('el arbolillo 2 croc','el arbolillo 2'),
('el arbolillo 3 croc','el arbolillo 3'),
('el milagro','la esmeralda'),
('el risco ctm','ctm el risco'),
('el tepetatal','tepetatal'),
('el tepetatal amp','tepetatal'),
('escuadron 2 01 infonavit','la laguna ticoman'),
('exescuela de tiro','ex escuela de tiro'),
('gabriel hernandez amp','gabriel hernandez'),
('gertrudis sanchez 1 secc','gertrudis sanchez 1 seccion'),
('gertrudis sanchez 2 secc','gertrudis sanchez 2 seccion'),
('gral felipe berriozabal','general felipe berriozabal'),
('granjas modernas','granjas modernas san juan de aragon amp'),
('guadalupe','guadalupe proletaria'),
('guadalupe ticoman','guadalupe tepeyac'),
('gustavo a madero','villa gustavo a madero'),
('industrial','industrial 1'),
('industrial vallejo','nueva industrial vallejo'),
('jose maria morelos y pavon 2','jose maria morelos y pavon 2'),
('juarez','santiago atzacoalco'),
('la escalera','residencia la escalera'),
('la forestal amp','la forestal 1'),
('la patera vallejo','la patera condomodulos')]

m_contreras = [('administracion barranca seca','barranca seca'),
('administracion padierna','heroes de padierna'),
('administracion san bernabe','lomas de san bernabe'),
('administracion san jeronimo lidice','san jeronimo aculco  lidice'),
('administracion unidad independencia','independencia san ramon'),
('atacasco','atacaxco'),
('centro de atencion al publico magdalena contreras','la magdalena atlitic'),
('chichicaspa amp','san jeronimo aculco  lidice'),
('crescencio juarez chavira','huayatla'),
('delegacion politica magdalena contreras','la magdalena atlitic'),
('el maestro','batan viejo (el maestro)'),
('el rosal bajo','el rosal'),
('ermitaño','el ermitaño'),
('fuentes del pedregal','la magdalena atlitic'),
('jardines en la montaña','la magdalena atlitic'),
('la magdalena contreras','la magdalena atlitic'),
('lomas de chamontoya','lomas de chamontoya')]

manuales = {'la magdalena contreras':m_contreras, 'iztapalapa':iztapalapa,'iztacalco':iztacalco,'gustavo a madero':g_a_madero}

In [12]:
# diccionarios a mano x alcaldia (j orcazas)
miguel_hidalgo = [('2 lagos','anahuac 2 lagos'),('abeto','lomas de bezares'),
    ('administracion anahuac','anahuac 1'),('administracion escandon','escandon 1'),
    ('anahuac','anahuac 1'),('argentina poniente','nueva argentina (argentina poniente)'),
    ('bosque de chapultepec parque nacional','bosques de chapultepec (polanco)'),
    ('bosques de chapultepec','bosques de chapultepec (polanco)'),('casa blanca','anzures'),
    ('centro de la ciudad de mexico area 1','polanco reforma (polanco)'),
    ('chapultepec morales','chapultepec morales (polanco)'),
    ('del bosque','polanco reforma (polanco)'),
    ('delegacion politica miguel hidalgo','polanco reforma (polanco)'),
    ('escandon','escandon 1'),('la loma','bosques de las lomas'),
    ('lago norte','anahuac lago norte'),('lago sur','anahuac lago sur'),
    ('las aguilas amp','lomas de sotelo'),
    ('lomas barrilaco secc vertientes','lomas de barrilaco (lomas de chapultepec)'),
    ('lomas de chapultepec','lomas de chapultepec 1'),('lomas de chapultepec 8 secc','lomas de chapultepec 3'),
    ('lomas del chamizal','lomas altas'),('lomas virreyes','lomas virreyes (lomas de chapultepec)'),
    ('los manzanos','anahuac los manzanos'),('los morales','polanco reforma (polanco)')]

milpa_alta = [('administracion milpa alta','santa ana tlacotenco'),('centro','san antonio tecomitl'),
    ('cruztitla','san antonio tecomitl'),('delegacion politica milpa alta','san antonio tecomitl'),
    ('la concepcion','villa milpa alta'),('la luz','villa milpa alta'),('los angeles','villa milpa alta'),
    ('san agustin','villa milpa alta')]

tlahuac = [('administracion la nopalera','la nopalera'),('administracion mixquic','san andres mixquic'),
    ('administracion tlahuac','la habana'),('aurorita','miguel hidalgo'),('cañada','miguel hidalgo'),
    ('cooperativa allapetlalli','del mar norte'),('cuatetlan','atotolco'),('del mar','del mar norte'),
    ('delegacion politica tlahuac','san jose'),('el rosario el alto','el rosario'),
    ('emiliano zapata 2 secc','emiliano zapata 2'),('guadalupe','san pedro tlahuac'),
    ('guadalupe tlaltenco','san francisco tlaltenco'),('guadalupe yecahuizotl','santa catarina yecahuizotl'),
    ('jardines del llano','san juan ixtayopan'),('jose lopez portillo','san francisco tlaltenco'),
    ('jose lopez portillo amp','san francisco tlaltenco'),('la asuncion','san jose'),
    ('la concepcion','santa catarina yecahuizotl'),('la conchita','la conchita 1')]

tlalpan = [('actopa','atocpa sur'),('administracion tlalpan','fuentes y pedregal de tepepan'),
    ('administracion villa coapa','granjas coapa'),('alvaro obregon','parques del pedregal'),
    ('america','tlalpan centro'),('ampliacion oriente','tlalpan centro'),
    ('arenal de guadalupe','arenal guadalupe tlalpan'),('arenal tepepan','arenal puerta tepepan'),
    ('belisario dominguez secc 1 6','belisario dominguez'),('belvedere de eslava','belvedere'),
    ('camisetas','la fama'),('cantera','cantera puente de piedra'),('chimalcoyotl','santa ursula xitla'),
    ('chimali residencial','arboledas del sur'),('club de golf mexico','club de golf mexico san buenaventura'),
    ('coapa','coapa villa cuemanco'),('colinas del ajusco','jardines del ajusco'),
    ('colinas del bosque','colinas del bosque las tortolas'),('cuemanco','coapa villa cuemanco'),
    ('cuevitas de curamaguey','barrio la lonja'),('cuitlahuac','jardines del ajusco'),
    ('cumbres de tepetongo','tepetongo'),('delegacion politica tlalpan','granjas coapa'),
    ('dr ignacio chavez infonavit','granjas coapa'),('ecuanil','ex hacienda san juan de dios'),
    ('ejidos de san andres','san andres totoltepec'),('el cantil','san miguel topilejo'),
    ('el capulin','miguel hidalgo'),('el charco','la primavera'),('el convento','santa ursula xitla'),
    ('el mirador','mirador 1 secc'),('el mirador 2 secc','mirador 2'),('el mirador 3 secc','mirador 2 y 3 secc'),
    ('el pedregal','parques del pedregal'),('estacion ajusco','mirador 2 y 3 secc'),
    ('estrella mora','parres el guarda'),('exejido de huipulco','arboledas del sur'),
    ('exejido de tepepan','san bartolo el chico'),('exhacienda coapa','jardines coapa belisario dominguez'),
    ('exhacienda san juan de dios','ex hacienda san juan de dios'),
    ('faroles del pedregal','parques del pedregal'),
    ('floresta coyoacan','floresta prado vergel coapa'),
    ('fuentes brotantes fovissste','fuentes brotantes miguel hidalgo'),
    ('fuentes brotantes parque nacional','fuentes brotantes miguel hidalgo'),
    ('fuentes de tepepan','fuentes y pedregal de tepepan'),
    ('gabriel ramos millan','coapa 2 seccion ramos millan'),
    ('guadalupe','hacienda san juan rincon de san juan chimali'),
    ('guadalupe tlalpan','hacienda san juan rincon de san juan chimali'),
    ('guadalupita','hacienda san juan rincon de san juan chimali'),
    ('hacienda de san juan','hacienda san juan rincon de san juan chimali'),
    ('hacienda de san juan de tlalpan 2 secc','hacienda san juan rincon de san juan chimali'),
    ('heroes de padierna','heroes de padierna 1'),('hueso periferico','granjas coapa')]

venustiano_carranza = [('2 0 de noviembre 5 tramo','5 tramo de 2 0 de noviembre'),
('administracion aeropuerto','el parque'),('administracion jamaica','el parque'),
('administracion via tapo','4 arboles'),('adolfo lopez mateos amp','adolfo lopez mateos'),
('aeropuerto internacional de la ciudad de mexico','el arenal 1 seccion'),
('arenal 1 secc','el arenal 1 seccion'),('arenal 2 secc','el arenal 2 seccion'),
('arenal 3 secc','el arenal 3 seccion'),('arenal 4 secc','el arenal 4 seccion'),
('bondojito','candelaria delos patos'),('candelaria de los patos fovissste','candelaria delos patos'),
('centro automatizado benito juarez','candelaria delos patos'),
('centro de la ciudad de mexico area 1','centro 1'),('centro de la ciudad de mexico area 8','centro 1'),
('delegacion politica venustiano carranza','kennedy'),('ignacio zaragoza','ignacio zaragoza 1'),
('jardin balbuena','jardin balbuena 1'),('juan polainas','jardin balbuena 1')]

xochimilco = ["xochimilco", ('abeto','san francisco tlalnepantla'),('acuezcomatl','san luis tlaxialtemalco'),
    ('administracion san mateo xalpa','san mateo xalpa'),('administracion tulyehualco','santiago tulyehualco'),
    ('administracion xochimilco','san bartolo el chico'),('arcos del sur','paseos del sur'),
    ('año de juarez','santa cruz acalpixca'),('belen','belem'),('bosques del sur','bosque residencial del sur'),
    ('bosques residencial del sur','bosque residencial del sur'),('calyequita','santiago tulyehualco'),
    ('cuatetlan','san mateo xalpa'),('del carmen','el carmen'),
    ('delegacion politica xochimilco','santa cecilia tepetlapa'),
    ('division del norte infonavit','santa cruz acalpixca'),('ejidos de tepepan','san lorenzo la cebada 1'),
    ('el arenal','san mateo xalpa'),('el cedral','guadalupe'),('el mirador','san marcos'),
    ('granjas coapa','la asuncion'),('jazmin','santa maria nativitas'),('la guadalupana','santa maria nativitas'),
    ('la herradura hueytepec','santa cecilia tepetlapa'),('las animas','santiago tulyehualco'),
    ('las mesitas','los cerrillos 2'),('lomas de tonalco','san lorenzo atemoaya'),
    ('los cerrillos','los cerrillos 1'),('maria g de garcia ruiz','santa cruz xochitepec'),
    ('mercado de flores plantas y hortalizas','la asuncion'),('misiones de la noria','san juan tepepan'),
    ('napoles amp','san pedro'),('nativitas amp','santiago tulyehualco'),('nativitas infonavit','xaltocan'),
    ('olivar','san luis tlaxialtemalco'),('oriente','santa cecilia tepetlapa'),
    ('paraje tlachiutepec','san mateo xalpa')]

manuales['miguel hidalgo'] = miguel_hidalgo
manuales['milpa alta'] = milpa_alta
manuales['tlahuac'] = tlahuac
manuales['tlalpan'] = tlalpan
manuales['venustiano carranza'] = venustiano_carranza
manuales['xochimilco'] = xochimilco

In [13]:
#diccionario a mano por alvlopzam
manuales['alvaro obregon'] = []
manuales['azcapotzalco'] = []
manuales['benito juarez'] = []
manuales['coyoacan'] = []
manuales['cuajimalpa de morelos'] = []

def corrige(viejo, nuevo, alcaldia):
    manuales[alcaldia].append((viejo,nuevo))

corrige('abeto', 'los cedros', 'alvaro obregon')
corrige('2 del moral', 'tetelpan', 'alvaro obregon')
corrige('2 del moral del pueblo tetelpan', 'tetelpan', 'alvaro obregon')
corrige('administracion observatorio', 'alpes amp', 'alvaro obregon')
corrige('administracion san angel', 'san angel', 'alvaro obregon')
corrige('aguila real', 'las aguilas', 'alvaro obregon')
corrige('alvaro obregon desarrollo urbano', 'el piru', 'alvaro obregon')
corrige('arcos del centenario', 'arcos de centenario', 'alvaro obregon')
corrige('axomiatla', 'lomas de axomiatla', 'alvaro obregon')
corrige('balcones de ceguayo', 'balcones de cehuayo', 'alvaro obregon')
corrige('belen', 'belen de las flores', 'alvaro obregon')
corrige('bellavista', 'bella vista', 'alvaro obregon')
corrige('bosques', '1 victoria seccion bosques', 'alvaro obregon')
corrige('bosques 2 secc', '1 victoria seccion bosques', 'alvaro obregon')
corrige('calzada de jalalpa', 'calzada jalalpa', 'alvaro obregon')
corrige('canutillo', 'canutillo(aguascalientes)', 'alvaro obregon')
corrige('canutillo 3 secc', 'canutillo 3 seccion', 'alvaro obregon')
corrige('carlos a madrazo', 'gral c a madrazo', 'alvaro obregon')
corrige('ceguaya', 'punta de cehuayo', 'alvaro obregon')
corrige('cooperativa ceguayo','cooperativa cehuayo', 'alvaro obregon')
corrige('corpus christi','corpus christy', 'alvaro obregon')
corrige('corpus christi 2 reacomodo','corpus christy', 'alvaro obregon')
corrige('corpus christi amp', 'corpus curisty ampl xocomecatla', 'alvaro obregon')
corrige('cuernito becerra reacomodo', 'reacomodo el cuernito', 'alvaro obregon')
corrige('cuevitas', 'las cuevitas', 'alvaro obregon')
corrige('desarrollo santa fe', 'santa fe', 'alvaro obregon')
corrige('el cuernito', 'reacomodo el cuernito', 'alvaro obregon')
corrige('el encino', 'tetelpan', 'alvaro obregon')
corrige('el pirul amp', '2 el pirul amp', 'alvaro obregon')
corrige('el tejocote', 'tejocote', 'alvaro obregon')
corrige('hidalgo', 'miguel hidalgo', 'alvaro obregon')
corrige('jalalpa 2 amp','2  jalalpa tepito amp', 'alvaro obregon')
corrige('jalalpa el grande reacomodo', 'jalalpa el grande', 'alvaro obregon')
corrige('jalalpatepito', 'jalalpa tepito', 'alvaro obregon')
corrige('jardines del pedregal de san angel', 'jardines del pedregal', 'alvaro obregon')
corrige('la cañada 1 secc','la cañada', 'alvaro obregon')
corrige('la cañada 2 secc','la cañada', 'alvaro obregon')
corrige('la cañada amp','la cañada', 'alvaro obregon')
corrige('la cebada', 'la cebada amp', 'alvaro obregon')
corrige('la esperanza o tlapechico 4 secc', 'tlapechico', 'alvaro obregon')
corrige('la presa secc hornos', 'la presa seccion hornos', 'alvaro obregon')
corrige('las aguilas amp 2 parque', 'las aguilas amp', 'alvaro obregon')
corrige('las aguilas amp 3 parque', 'aguilas 3 parque', 'alvaro obregon')
corrige('las cuevitas reacomodo', 'las cuevitas', 'alvaro obregon')
corrige('las golondrinas', 'golondrinas', 'alvaro obregon')
corrige('las golondrinas 2 secc', 'golondrinas 2 seccion', 'alvaro obregon')
corrige('lomas de becerra granada', 'lomas de becerra', 'alvaro obregon')
corrige('lomas de la hera', 'lomas de la era', 'alvaro obregon')
corrige('administracion azcapotzalco', 'sindicato mexicano de electricistas', 'azcapotzalco')
corrige('azcapotzalco', 'del recreo', 'azcapotzalco')
corrige('azcapotzalco centro', 'centro de azcapotzalco', 'azcapotzalco')
corrige('central rosario', 'nueva el rosario', 'azcapotzalco')
corrige('delegacion politica azcapotzalco', 'petrolera', 'azcapotzalco')
corrige('ejido el rosario', 'san martin xochinahuac', 'azcapotzalco')
corrige('el jaguey','el jaguey-estacion pantaco', 'azcapotzalco')
corrige('el recreo', 'del recreo', 'azcapotzalco')
corrige('el rosario infonavit', 'el rosario', 'azcapotzalco')
corrige('electricistas', 'sindicato mexicano de electricistas', 'azcapotzalco')
corrige('hacienda del rosario', 'ex-hacienda el rosario', 'azcapotzalco')
corrige('hogar y seguridad', 'hogar y seguridad/nueva santa maria', 'azcapotzalco')
corrige('las trancas', 'miguel hidalgo', 'azcapotzalco')
corrige('loteria nacional', 'claveria', 'azcapotzalco')
corrige('pdte francisco 1 madero', 'presidente madero', 'azcapotzalco')
corrige('petrolera nueva amp', 'petrolera amp', 'azcapotzalco')
corrige('prohogar', 'pro hogar 1', 'azcapotzalco')
corrige('san antonio amp', 'san antonio', 'azcapotzalco')
corrige('san bartolo cacahualtongo', 'san bartolo cahualtongo', 'azcapotzalco')
corrige('san pablo', 'san pablo xalpa', 'azcapotzalco')
corrige('administracion colonia del valle', 'del valle 1', 'benito juarez')
corrige('administracion municipio libre', 'sta cruz atoyac', 'benito juarez')
corrige('alamos','alamos 1', 'benito juarez')
corrige('americas unidas', 'americas unidas-del lago', 'benito juarez')
corrige('centro urbano pdte aleman', 'centro urbano presidente aleman', 'benito juarez')
corrige('del lago', 'americas unidas-del lago', 'benito juarez')
corrige('del periodista francisco zarco', 'periodista francisco zarco', 'benito juarez')
corrige('del valle', 'del valle 1', 'benito juarez')
corrige('del valle norte', 'del valle 1', 'benito juarez')
corrige('del valle norte', 'del valle 2', 'benito juarez')
corrige('delegacion politica benito juarez', 'sta cruz atoyac', 'benito juarez')
corrige('emperadores', 'residencial emperadores', 'benito juarez')
corrige('esperanza', 'narvarte 5', 'benito juarez')
corrige('gral pedro maria anaya','general pedro maria anaya', 'benito juarez')
corrige('mail boxes ramon a gonzalez gutierrez','napoles', 'benito juarez')
corrige('acasulco','altillo (cond altillo acasulco)', 'coyoacan')
corrige('administracion camionera del sur', 'atlantida', 'coyoacan')
corrige('administracion coyoacan', 'la concepcion', 'coyoacan')
corrige('administracion fovissste', 'copilco universidad', 'coyoacan')
corrige('adolfo ruiz cortines', 'adolfo ruiz cortines 1', 'coyoacan')
corrige('ajusco', 'ajusco 1', 'coyoacan')
corrige('ajusco montserrat', 'ajusco 1', 'coyoacan')
corrige('alianza popular revolucionaria fovissste', 'alianza popular revolucionaria poniente', 'coyoacan')
corrige('altillo universidad','altillo (cond altillo universidad)', 'coyoacan')
corrige('cafetales','cafetales 1 (rdcial)', 'coyoacan')
corrige('churubusco','san diego churubusco', 'coyoacan')
corrige('churubusco country club','country club', 'coyoacan')
corrige('copilco', 'copilco universidad', 'coyoacan')
corrige('copilco universidad issste', 'copilco universidad', 'coyoacan')
corrige('coyoacan centro', 'centro urbano', 'coyoacan')
corrige('cuchilla de la magdalena', 'la magdalena culhuacan', 'coyoacan')
corrige('culhuacan ctm obrero secc 1 0','ctm 1 0 culhuacan', 'coyoacan')
corrige('culhuacan ctm obrero secc 5','ctm 5 culhuacan', 'coyoacan')
corrige('culhuacan ctm obrero secc 6','ctm 6 culhuacan', 'coyoacan')
corrige('culhuacan ctm obrero secc 7','ctm 7 culhuacan', 'coyoacan')
corrige('culhuacan ctm obrero secc 8','ctm 8 culhuacan', 'coyoacan')
corrige('culhuacan ctm obrero secc 9','ctm 9 culhuacan zona 2 9 -30', 'coyoacan')
corrige('culhuacan piloto 5', 'piloto culhuacan', 'coyoacan')
corrige('del carmen amp', 'del carmen', 'coyoacan')
corrige('el hueso infonavit','el vergel de coyoacan ( infonavit el hueso)', 'coyoacan')
corrige('el parque','el parque de coyoacan', 'coyoacan')
corrige('el rosedal', 'el rosedal 1', 'coyoacan')
corrige('exejido san pablo tepetlapa', 'san pablo tepetlapa', 'coyoacan')
corrige('exejido santa ursula coapa', 'viejo ejido santa ursula coapa', 'coyoacan')
corrige('exhacienda coapa', 'ex hacienda coapa', 'coyoacan')
corrige('fortin de chimalistac', 'chimalistac', 'coyoacan')
corrige('hacienda de coyoacan', 'haciendas de coyoacan', 'coyoacan')
corrige('huayamilpas','ajusco huayamilpas', 'coyoacan')
corrige('insurgentes cuicuilco amp', 'insurgentes cuicuilco', 'coyoacan')
corrige('insurgentes san angel', 'copilco el bajo', 'coyoacan')
corrige('integracion latino americana','integracion latinoamericana', 'coyoacan')
corrige('administracion contadero','el contadero', 'cuajimalpa de morelos')
corrige('altiburritos', 'cruz blanca', 'cuajimalpa de morelos')
corrige('bosque de la reforma prolongacion','san pedro cuajimalpa', 'cuajimalpa de morelos')
corrige('campestre palo alto','palo alto (granjas)', 'cuajimalpa de morelos')
corrige('cooperativa palo alto', 'palo alto (granjas)', 'cuajimalpa de morelos')
corrige('cruz manca', 'el contadero', 'cuajimalpa de morelos')
corrige('cuajimalpa', 'san pedro cuajimalpa', 'cuajimalpa de morelos')
corrige('delegacion politica cuajimalpa de morelos', 'san pedro cuajimalpa', 'cuajimalpa de morelos')
corrige('doña juana', 'el contadero', 'cuajimalpa de morelos')
corrige('el ebano', 'ebano', 'cuajimalpa de morelos')
corrige('el puente', 'san pedro cuajimalpa', 'cuajimalpa de morelos')
corrige('el zarco', 'cola de pato', 'cuajimalpa de morelos')
corrige('granjas de palo alto', 'palo alto (granjas)', 'cuajimalpa de morelos')

In [14]:
len_manuales = 0
for alcaldia in manuales:
    for colonias in manuales[alcaldia]:
        len_manuales = len_manuales +1

len_manuales

438

In [15]:
# vamos a utilizar levenstein entre las colonias nuevas y las de colonias_pob para 
# maximizar el uso del diccionario manual

#creamos dataframe con las colonias nuevas manuales
colonias_manuales = pd.DataFrame(columns = ['alcaldia', 'colonia_vieja','colonia_nueva'], index = range(len_manuales))

index = 0
for alcaldia in manuales:
    for colonias in manuales[alcaldia]:
        colonias_manuales.iloc[index, 0] = alcaldia
        colonias_manuales.iloc[index, 1] = colonias[0]
        colonias_manuales.iloc[index, 2] = colonias[1]
        index = index + 1

#hacemos levenstein
indexer = rl.Index()
indexer.full()
pairs = indexer.index(colonias_manuales, colonias_pob)

compare_cl = rl.Compare()
compare_cl.string("colonia_nueva", "colonia", threshold=.75, label = 'colonia')
compare_cl.exact("alcaldia", "alcaldia", label='alcaldia')
features = compare_cl.compute(pairs, colonias_manuales, colonias_pob)

features[(features['colonia']==1)&(features['alcaldia']==1)]

#actualizamos colonias_manuales

parejas = features[(features['colonia']==1)&(features['alcaldia']==1)].index

for p in parejas:
    colonias_manuales.iloc[p[0], 2] = colonias_pob.iloc[p[1], 1]


In [16]:
#se aplican colonias manuales (actualizar colonias_matched e index_faltantes_paot)

for i in range(colonias_manuales.shape[0]):

    ind_paot = colonias_paot.index[colonias_paot['colonia']==colonias_manuales.iloc[i, 1]]
    

    if((len(ind_paot)==1) & (colonias_manuales.iloc[i, 2] in colonias_pob['colonia'].to_list())):

        if(ind_paot[0] in index_faltantes_paot):
            index_faltantes_paot.remove(ind_paot[0])
            
        colonias_matched.iloc[ind_paot[0], 0] = colonias_manuales.iloc[i, 0]
        colonias_matched.iloc[ind_paot[0], 1] = colonias_manuales.iloc[i, 1]
        colonias_matched.iloc[ind_paot[0], 2] = colonias_manuales.iloc[i, 2]
        

colonias_matched.dropna()
            


,alcaldia,colonia_vieja,colonia_nueva
0,iztacalco,agricola oriental,agricola oriental 8
2,iztacalco,administracion apatlaco,agricola oriental 8
3,iztacalco,gabriel ramos millan amp,tlacotal ramos millan
4,iztacalco,gabriel ramos millan,gabriel ramos millan
6,iztacalco,agricola pantitlan,agricola oriental 8
...,...,...,...
1885,xochimilco,el cedral,guadalupe
1887,xochimilco,san jose obrero,san jose obrero
1888,xochimilco,arcos del sur,paseos del sur
1890,xochimilco,la guadalupana,santa maria nativitas pblo


In [17]:
#ahora utilicemos la distancia de levenstein para encontrar mas matches

colonias_paot_faltantes = colonias_paot[colonias_paot.index.isin(index_faltantes_paot)]

indexer = rl.Index()
indexer.full()
pairs = indexer.index(colonias_pob, colonias_paot_faltantes)

compare_cl = rl.Compare()
compare_cl.string("colonia", "colonia", threshold=.75, label = 'colonia')
compare_cl.exact("alcaldia", "alcaldia", label='alcaldia')
features = compare_cl.compute(pairs, colonias_pob, colonias_paot_faltantes)

features[(features['colonia']==1)&(features['alcaldia']==1)]


,,colonia,alcaldia
7,1259,1.0,1
11,1230,1.0,1
13,1197,1.0,1
25,1210,1.0,1
45,1234,1.0,1
...,...,...,...
1792,1333,1.0,1
1793,1272,1.0,1
1794,1272,1.0,1
1805,1334,1.0,1


In [18]:
#actualizamos matched y colonias faltantes

parejas = features[(features['colonia']==1)&(features['alcaldia']==1)].index

for p in parejas:
    if p[1] in index_faltantes_paot:
        index_faltantes_paot.remove(p[1])
    
    colonias_matched.iloc[p[1], 0] = colonias_paot.iloc[p[1], 0]
    colonias_matched.iloc[p[1], 1] = colonias_paot.iloc[p[1], 1]
    colonias_matched.iloc[p[1], 2] = colonias_pob.iloc[p[0], 1]


In [19]:
#hasta aqui llegamos, companierxs, ahora toca armar el resto manual :$ (recuerden q esto es por el bien del analisis q viene)
colonias_matched[colonias_matched['colonia_nueva'].isin(colonias_pob['colonia'])]
colonias_matched[colonias_matched['colonia_vieja'].isin(colonias_paot['colonia'])]

colonias_matched.dropna().head(50)

#que todos estos dataframes equivalgan explica que el linkage hasta ahora va bien. 


,alcaldia,colonia_vieja,colonia_nueva
0,iztacalco,agricola oriental,agricola oriental 8
2,iztacalco,administracion apatlaco,agricola oriental 8
3,iztacalco,gabriel ramos millan amp,tlacotal ramos millan
4,iztacalco,gabriel ramos millan,gabriel ramos millan
6,iztacalco,agricola pantitlan,agricola oriental 8
7,iztacalco,juventino rosas,juventino rosas 2
8,iztacalco,santa anita,santa anita
10,iztacalco,reforma iztaccihuatl norte,reforma iztaccihuatl norte
11,iztacalco,cuchilla ramos millan,cuchilla ramos millan
12,iztacalco,granjas mexico,granjas mexico 2


In [20]:
#preparamos talacha:
colonias_paot[colonias_paot.index.isin(index_faltantes_paot)].to_excel(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\raw\las_que_faltan.xlsx')
colonias_pob.to_excel(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\raw\colonias_pob.xlsx')

In [21]:
colonias_matched.dropna()

#len(index_faltantes_paot)

,alcaldia,colonia_vieja,colonia_nueva
0,iztacalco,agricola oriental,agricola oriental 8
2,iztacalco,administracion apatlaco,agricola oriental 8
3,iztacalco,gabriel ramos millan amp,tlacotal ramos millan
4,iztacalco,gabriel ramos millan,gabriel ramos millan
6,iztacalco,agricola pantitlan,agricola oriental 8
...,...,...,...
1885,xochimilco,el cedral,guadalupe
1887,xochimilco,san jose obrero,san jose obrero
1888,xochimilco,arcos del sur,paseos del sur
1890,xochimilco,la guadalupana,santa maria nativitas pblo


In [22]:
las_que_faltan = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\raw\las_que_faltan.xlsx - Sheet1 (4).csv', encoding = 'latin-1')

In [23]:
las_que_faltan_sin_na = las_que_faltan.dropna()

index = 0
for i in range(las_que_faltan_sin_na.shape[0]):

    colonias_matched.iloc[las_que_faltan_sin_na.iloc[index,0], 0] = las_que_faltan_sin_na.iloc[index,1]
    colonias_matched.iloc[las_que_faltan_sin_na.iloc[index,0], 1] = las_que_faltan_sin_na.iloc[index,2]
    colonias_matched.iloc[las_que_faltan_sin_na.iloc[index,0], 2] = las_que_faltan_sin_na.iloc[index,3]

    index = index +1


#colonias_matched[colonias_matched['alcaldia']=='iztacalco'].head(50)

#colonias_paot[colonias_paot['alcaldia']=='iztacalco']

#colonias_matched.dropna()

In [24]:
colonias_matched.dropna

<bound method DataFrame.dropna of         alcaldia                           colonia_vieja  \
0      iztacalco                       agricola oriental   
1      iztacalco                               los reyes   
2      iztacalco                 administracion apatlaco   
3      iztacalco                gabriel ramos millan amp   
4      iztacalco                    gabriel ramos millan   
...          ...                                     ...   
1888  xochimilco                           arcos del sur   
1889  xochimilco  mercado de flores plantas y hortalizas   
1890  xochimilco                          la guadalupana   
1891  xochimilco                             tecacalanco   
1892  xochimilco                                san jose   

                   colonia_nueva  
0            agricola oriental 8  
1                 los reyes barr  
2            agricola oriental 8  
3          tlacotal ramos millan  
4           gabriel ramos millan  
...                          ...  


In [25]:
arreglar_p2 = colonias_matched.loc[~colonias_matched['colonia_nueva'].isin(colonias_pob['colonia'])].dropna()
#colonias_matched[colonias_matched['colonia_vieja'].isin(colonias_paot['colonia'])]

#colonias_matched.dropna()

arreglar_p2.shape[0]

124

In [26]:
#empezamos primer filtro: linkage con matches exactos en colonia y alcaldia
#para entender esto recomendamos leer documentacion de recordlinkage

indexer = rl.Index()
indexer.full()
pairs = indexer.index(arreglar_p2, colonias_pob)

compare_cl = rl.Compare()
compare_cl.string("colonia_nueva", "colonia",threshold=.7, label = 'colonia')
compare_cl.exact("alcaldia", "alcaldia", label='alcaldia')
features = compare_cl.compute(pairs, arreglar_p2, colonias_pob)

features[(features['colonia']==1)&(features['alcaldia']==1)]

,,colonia,alcaldia
53,1698,1.0,1
60,1720,1.0,1
62,1725,1.0,1
65,1704,1.0,1
70,1725,1.0,1
...,...,...,...
1812,1618,1.0,1
1817,1643,1.0,1
1825,1652,1.0,1
1843,1622,1.0,1


In [27]:
indices = features[(features['colonia']==1)&(features['alcaldia']==1)].index.tolist()

for i in indices:
    colonias_matched.iloc[i[0], 2] = colonias_pob.iloc[i[1],1]


In [28]:
arreglar_p2 = colonias_matched.loc[~colonias_matched['colonia_nueva'].isin(colonias_pob['colonia'])].dropna()

indexer = rl.Index()
indexer.full()
pairs = indexer.index(arreglar_p2, colonias_pob)

compare_cl = rl.Compare()
compare_cl.string("colonia_nueva", "colonia",threshold=.6, label = 'colonia')
compare_cl.exact("alcaldia", "alcaldia", label='alcaldia')
features = compare_cl.compute(pairs, arreglar_p2, colonias_pob)

features[(features['colonia']==1)&(features['alcaldia']==1)].shape[0]

indices = features[(features['colonia']==1)&(features['alcaldia']==1)].index.tolist()

for i in indices:
    colonias_matched.iloc[i[0], 2] = colonias_pob.iloc[i[1],1]


In [29]:
colonias_matched.loc[~colonias_matched['colonia_nueva'].isin(colonias_pob['colonia'])].dropna().sort_values('alcaldia')



,alcaldia,colonia_vieja,colonia_nueva
359,alvaro obregon,sin colonia,sin colonia
393,alvaro obregon,san jeronimo lidice,san jeronimo lidice
410,alvaro obregon,centro de la ciudad de mexico area 1,centro de la ciudad de mexico area 1
1255,azcapotzalco,sin colonia,sin colonia
1145,benito juarez,sin colonia,sin colonia
1659,coyoacan,sin colonia,sin colonia
1073,cuajimalpa de morelos,lomas de chapultepec,lomas de chapultepec
1058,cuajimalpa de morelos,sin colonia,sin colonia
724,miguel hidalgo,loma hermosa,loma hermosa
730,miguel hidalgo,sin colonia,sin colonia


In [30]:
colonias_matched.iloc[359,2] = 'san pedro de los pinos'
colonias_matched.iloc[393,2] = 'arvide'
colonias_matched.iloc[410,2] = 'alpes'
colonias_matched.iloc[1255,2] = 'centro de azcapotzalco'
colonias_matched.iloc[1145,2] = 'acacias'
colonias_matched.iloc[1659,2] = 'del carmen'
colonias_matched.iloc[1073,2] = 'lomas del chamizal'
colonias_matched.iloc[1058,2] = 'lomas del chamizal'
colonias_matched.iloc[724,2] = 'loma hermosa conj hab'
colonias_matched.iloc[730,2] = 'lomas de chapultepec 3'
colonias_matched.iloc[1692,2] = 'san jeronimo miacatlan pblo'
colonias_matched.iloc[51,2] = 'la conchita 2'
colonias_matched.iloc[120, 2] = 'la turba'
colonias_matched.iloc[555, 2] = 'nino jesus barr'
colonias_matched.iloc[516, 2] = 'tlalpan centro'
colonias_matched.iloc[575, 2] = 'tlalpan centro'
colonias_matched.iloc[1282, 2] = 'kennedy u hab'
colonias_matched.iloc[1328, 2] = 'bahia u hab'
colonias_matched.iloc[1338, 2] = 'bahia u hab'
colonias_matched.iloc[1344, 2] = 'kennedy u hab'
colonias_matched.iloc[1839, 2] = 'san marcos barr'
colonias_matched.iloc[1795, 2] = 'villa xochimilco u hab'
colonias_matched.iloc[1818, 2] = 'nativitas'
colonias_matched.iloc[1819, 2] = 'belem barr'
colonias_matched.iloc[1875, 2] = 'cerro grande'



In [31]:
colonias_matched.loc[~colonias_matched['colonia_nueva'].isin(colonias_pob['colonia'])].dropna()

,alcaldia,colonia_vieja,colonia_nueva
1820,xochimilco,tlalpuente,tlalpuente
1868,xochimilco,san juan de los lagos,sin colonia
1882,xochimilco,rinconada coapa 2 secc,sin colonia
1884,xochimilco,maria g de garcia ruiz,sin colonia
1889,xochimilco,mercado de flores plantas y hortalizas,sin colonia


In [32]:
colonias_matched.dropna()

,alcaldia,colonia_vieja,colonia_nueva
0,iztacalco,agricola oriental,agricola oriental 8
1,iztacalco,los reyes,los reyes barr
2,iztacalco,administracion apatlaco,agricola oriental 8
3,iztacalco,gabriel ramos millan amp,tlacotal ramos millan
4,iztacalco,gabriel ramos millan,gabriel ramos millan
...,...,...,...
1888,xochimilco,arcos del sur,paseos del sur
1889,xochimilco,mercado de flores plantas y hortalizas,sin colonia
1890,xochimilco,la guadalupana,santa maria nativitas pblo
1891,xochimilco,tecacalanco,tecacalanco


In [33]:
colonias_matched.to_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\colonias_matched.csv')

In [34]:
colonias_matched[colonias_matched['colonia_nueva'].isin(colonias_pob['colonia'])]
#colonias_matched[colonias_matched['colonia_vieja'].isin(colonias_paot['colonia'])]

#colonias_matched.dropna()

,alcaldia,colonia_vieja,colonia_nueva
0,iztacalco,agricola oriental,agricola oriental 8
1,iztacalco,los reyes,los reyes barr
2,iztacalco,administracion apatlaco,agricola oriental 8
3,iztacalco,gabriel ramos millan amp,tlacotal ramos millan
4,iztacalco,gabriel ramos millan,gabriel ramos millan
...,...,...,...
1887,xochimilco,san jose obrero,san jose obrero
1888,xochimilco,arcos del sur,paseos del sur
1890,xochimilco,la guadalupana,santa maria nativitas pblo
1891,xochimilco,tecacalanco,tecacalanco


In [35]:

funciones.estandarizacion(df_pob, 'alcaldia', 'colonia')

df_pob

,cve_ent,alcaldia,cve_col,colonia,pob_2010,SUP_COL_M2,VIV2010,DENVIVHa,SUPMNZHa,C_DENVIVHa
0,09,azcapotzalco,02-001,aguilera,2014.0,103825.0,680,65.4948,10.38250,4
1,09,azcapotzalco,02-002,aldana,3378.0,154812.0,1088,70.2788,15.48120,4
2,09,azcapotzalco,02-005,angel zimbron,2737.0,242232.0,1067,44.0487,24.22320,3
3,09,azcapotzalco,02-006,arenal,4817.0,286075.0,1493,52.1891,28.60750,3
4,09,azcapotzalco,02-007,centro de azcapotzalco,3043.0,424471.0,1118,26.3387,42.44710,2
...,...,...,...,...,...,...,...,...,...,...
1809,09,alvaro obregon,10-234,union popular emiliano zapata u hab,2618.0,137210.0,796,58.0133,13.72100,3
1810,09,alvaro obregon,10-235,valentin gomez farias,1835.0,113947.0,416,36.5082,11.39470,2
1811,09,alvaro obregon,10-237,villa solidaridad,588.0,30095.5,189,62.8001,3.00955,4
1812,09,alvaro obregon,10-239,zenon delgado,2658.0,107451.0,720,67.0073,10.74510,4


In [36]:
colonias_matched['cve_col'] = None

for i in range(colonias_matched.shape[0]):
    datos_temp = colonias_pob[(colonias_pob['alcaldia']==colonias_matched.iloc[i,0]) & (colonias_pob['colonia'] == colonias_matched.iloc[i,2])]
    if(not datos_temp.empty):
        colonias_matched.iloc[i, 3] = datos_temp.iloc[0,2]

 

colonias_matched.head(50)

,alcaldia,colonia_vieja,colonia_nueva,cve_col
0,iztacalco,agricola oriental,agricola oriental 8,06-047
1,iztacalco,los reyes,los reyes barr,06-018
2,iztacalco,administracion apatlaco,agricola oriental 8,06-047
3,iztacalco,gabriel ramos millan amp,tlacotal ramos millan,06-036
4,iztacalco,gabriel ramos millan,gabriel ramos millan,06-010
5,iztacalco,iztacalco infonavit,infonavit iztacalco u hab 2,06-053
6,iztacalco,agricola pantitlan,agricola oriental 8,06-047
7,iztacalco,juventino rosas,juventino rosas 2,06-055
8,iztacalco,santa anita,santa anita,06-032
9,iztacalco,san pedro,san pedro iztacalco barr,06-031


In [37]:
colonias_matched.to_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\colonias_matched.csv')

In [45]:
colonias_matched[colonias_matched['colonia_nueva'].isin(colonias_pob['colonia'])]
colonias_matched[colonias_matched['colonia_vieja'].isin(colonias_paot['colonia'])]

colonias_matched[colonias_matched['cve_col'].isin(colonias_pob['cve_col'])]['cve_col'].unique().size

#colonias_matched['colonia_nueva'].unique().size

1286